In [111]:
from ctr.importxml import *
import lxml
from bs4 import BeautifulSoup

In [114]:
p = 'F:\\Reporting\\CTR\\2021\\September'
path = Path(p)
file_list = get_file_list(path)
xml_list = get_xml_list(file_list)
merged_xml = merge_xml(xml_list)
transaction_list = merged_xml.findall('transaction')

In [115]:
tr = transaction_list[0]

In [133]:
location = tr.find('transaction_location').text
description = tr.find('transaction_description').text
date = tr.find('date_transaction').text
mode = tr.find('transmode_code').text
amount = tr.find('amount_local').text

In [134]:
location, description, date, mode, amount

('PRINCIPAL BRANCH,DHAKA,NRB Commercial Bank Limited',
 'Cash Withdrawl',
 '2021-09-14T00:00:00',
 'R',
 '1100000')

In [74]:
tr = merged_xml.find('transactionnumber')

In [77]:
merged_xml

<Element 'report' at 0x000002DA004D7EA8>